## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

In [ ]:
def canny_edge_detection(image, high_threshold, low_threshold):
    X, Y = image.shape

    high_threshold_mask = image >= high_threshold
    low_threshold_mask = np.where(np.logical_and(high_threshold > image, image >= low_threshold), 1, 0)

    return high_threshold_mask, low_threshold_mask

def nonmax(directions, magnitude):
    X, Y = directions.shape
    nonmax_image = np.zeros((X, Y))

    for i in range(1, X - 1):
        for j in range(1, Y - 1):
            if directions[i, j] == 1:
                if magnitude[i, j - 1] > magnitude[i, j] or magnitude[i, j + 1] > magnitude[i, j]:
                    nonmax_image[i, j] = 0
                else:
                    nonmax_image[i, j] = magnitude[i, j]
            elif directions[i, j] == 2:
                if magnitude[i + 1, j - 1] > magnitude[i, j] or magnitude[i - 1, j + 1] > magnitude[i, j]:
                    nonmax_image[i, j] = 0
                else:
                    nonmax_image[i, j] = magnitude[i, j]
            elif directions[i, j] == 3:
                if magnitude[i + 1, j] > magnitude[i, j] or magnitude[i - 1, j] > magnitude[i, j]:
                    nonmax_image[i, j] = 0
                else:
                    nonmax_image[i, j] = magnitude[i, j]
            elif directions[i, j] == 4:
                if magnitude[i - 1, j - 1] > magnitude[i, j] or magnitude[i + 1, j + 1] > magnitude[i, j]:
                    nonmax_image[i, j] = 0
                else:
                    nonmax_image[i, j] = magnitude[i, j]

    return nonmax_image

def canny(img, th, tl):
    X, Y = img.shape
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)
    Sobel1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Sobel2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    Gx = cv2.filter2D(img_blur, -1, Sobel1)
    Gy = cv2.filter2D(img_blur, -1, Sobel2)
    magnitude = np.sqrt(Gx**2 + Gy**2)
    angle = np.arctan2(Gy, Gx)
    angle_deg = angle * 180. / np.pi
    angle_deg[angle_deg < 0] += 180
    directions = np.zeros((X, Y))

    for i in range(X):
        for j in range(Y):
            if (0 <= angle_deg[i, j] < 22.5) or (157.5 <= angle_deg[i, j] <= 180):
                directions[i, j] = 1
            elif (22.5 <= angle_deg[i, j] < 67.5):
                directions[i, j] = 2
            elif (67.5 <= angle_deg[i, j] < 112.5):
                directions[i, j] = 3
            elif (112.5 <= angle_deg[i, j] < 157.5):
                directions[i, j] = 4

    nonmax_image = nonmax(directions, magnitude)
    gnh, gnl = canny_edge_detection(nonmax_image, th, tl)

    return gnh, gnl

def connect_edges(gnh, gnl, image):
    height, width = image.shape
    edge_pixels = []

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            if gnh[i, j] == 1:
                edge_pixels.append([i, j])

    visited = np.zeros((height, width))

    connected_edges = np.zeros((height, width))

    while edge_pixels:
        current_pixel = edge_pixels.pop()
        x, y = current_pixel

        if visited[x, y] == 0:
            visited[x, y] = 1
            connected_edges[x, y] = 1

            for i in range(x - 1, x + 2):
                for j in range(y - 1, y + 2):
                    if gnl[i, j] == 1:
                        edge_pixels.append([i, j])

    return connected_edges

In [ ]:
I_dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
gnh,gnl = canny(I_dom, 5, 10)
image_canny = connect_edges(gnh, gnl, I_dom)
canny_opencv = cv2.Canny(I_dom, 5, 10, None, 3, L2gradient=True)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,16))

ax1.set_title("Original")
ax1.imshow(I_dom, 'gray')
ax1.axis('off')

ax2.set_title("GNH")
ax2.imshow(gnh, 'gray')
ax2.axis('off')

ax3.set_title("Detected Edges (Custom Canny)")
ax3.imshow(image_canny, 'gray')
ax3.axis('off')

ax4.set_title("Detected Edges (OpenCV Canny)")
ax4.imshow(canny_opencv, 'gray')
ax4.axis('off')

plt.show()